# 3. Pré-processamento

Nosso modelo só pode prever os jogos com base em dados passados dos times, não a partir das variáveis do jogo a ser previsto (até porque a previsão ocorre antes do jogo). O algoritmo deve se basear apenas nas informações passadas

In [3]:
import pandas as pd
import datetime as dt
import numpy as np
import os

In [4]:
df = pd.read_csv('dados_brasileirao_tratados.csv') 
brasileirao = pd.read_csv('jogos_brasileirao.csv')

## Calcular média móvel

Para uma análise de resultado normalmente leva-se em conta os últimos 3 ou 5 jogos disputados pelas equipes. Por conta disso, calcularemos a **média móvel dos últimos 5 jogos de cada time**

### Juntando dataframes

Vou precisar atribuir o ano dos jogos às linhas na base de dados usada para fazer o treinamento. Isso porque terei de agrupar por ano os dados para calcular a média móvel

In [10]:
# Criando uma coluna 'ano' com o ano em que cada jogo aconteceu
try:
    brasileirao['ano'] = pd.to_datetime(brasileirao['data'].str.title(), dayfirst=True).dt.year
    
except AttributeError:
    print('A conversão já foi feita')

In [11]:
brasileirao.rename(columns={'ID': 'partida_id'}, inplace=True)

In [12]:
#Fazendo a junção dos dataframes
df_atualizado = pd.merge(brasileirao[['partida_id', 'ano']], df, on='partida_id')

In [13]:
df_atualizado

,partida_id,ano,rodata,clube,chutes,chutes_no_alvo,posse_de_bola,passes,precisao_passes,faltas,cartao_amarelo,cartao_vermelho,impedimentos,escanteios
0,6126,2018,1,Cruzeiro,12.0,2.0,0.40,425.0,0.81,20.0,3,0,0,7
1,6126,2018,1,Gremio,6.0,2.0,0.60,649.0,0.86,15.0,1,1,3,4
2,6127,2018,1,Vitoria,21.0,6.0,0.66,545.0,0.88,24.0,3,0,2,4
3,6127,2018,1,Flamengo,10.0,5.0,0.34,286.0,0.72,10.0,2,1,2,5
4,6128,2018,1,Santos,22.0,3.0,0.54,532.0,0.90,7.0,0,0,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3795,8023,2022,38,Atletico-MG,17.0,7.0,0.41,351.0,0.82,19.0,6,0,0,4
3796,8024,2022,38,Palmeiras,11.0,2.0,0.52,306.0,0.82,17.0,3,0,2,5
3797,8024,2022,38,Internacional,15.0,8.0,0.48,287.0,0.77,13.0,2,0,1,6
3798,8025,2022,38,Goias,6.0,2.0,0.35,330.0,0.80,12.0,2,0,1,3


### Calculando a média móvel

Com a base de dados devidamente ajustada, podemos calcular a média móvel dos últimos 5 jogos de cada time

In [79]:
#Cálculo da média móvel
df_mm = df_atualizado.groupby(['ano', 'clube'])[df_atualizado.iloc[:, 4:].columns].rolling(5, min_periods=1).mean()

#Defasagem de variáveis
df_mm = df_mm[df_mm.columns.difference(['ano', 'clube'])].groupby(['ano', 'clube']).shift(1).reset_index().fillna(0)

df_mm = df_mm.rename(columns = {'level_2':'index'})

#Retorna os valores para a base original
df_atualizado = df_atualizado.reset_index().iloc[:, :5].merge(df_mm)

## Aproveitamento

#### Ajeitar dataframe

Agora precisamos calcular o aproveitamento dos times. Para tal, precisamos dividir o número de pontos obtido pelo número de pontos possíveis de serem conquistados até aquele momento

Terei de criar um novo dataframe para juntar com o *'df_atualizado'*. Devido ao número incompatível de colunas terei de realocar a coluna 'visitante' abaixo da 'mandante' e assim colar as duas

In [24]:
#Criar variável binária para identificar se um time está jogando em casa ou fora

brasileirao.loc[brasileirao['vencedor'] == brasileirao['mandante'], 'casa'] = 1
brasileirao.loc[brasileirao['vencedor'] != brasileirao['mandante'], 'casa'] = 0
brasileirao['casa'] = brasileirao['casa'].astype(int)

#Iniciar a coluna 'pontos'
brasileirao['pontos'] = 0

#Calcular os pontos conquistados pelos times
brasileirao.loc[(brasileirao['resultado'] == 0) & (brasileirao['casa'] == 1), 'pontos'] = 3
brasileirao.loc[(brasileirao['resultado'] == 1) & (brasileirao['casa'] == 0), 'pontos'] = 3
brasileirao.loc[(brasileirao['resultado'] == 2), 'pontos'] = 1

**A partir daqui criou-se o dataframe *df_final***

In [67]:
colunas = brasileirao[['mandante', 'visitante', 'vencedor', 'resultado', 'ano', 'rodata', 'mandante_Placar', 'visitante_Placar']]

mandantes = colunas['mandante']
visitantes = colunas['visitante']
resultado = colunas['resultado']
ano = colunas['ano']
rodada = colunas['rodata']
mandante_placar = colunas['mandante_Placar']
visitante_placar = colunas['visitante_Placar']

times_alternados = pd.concat([mandantes, visitantes]).sort_index(kind='merge').reset_index(drop=True)

vencedores_binarios = pd.concat([(mandantes == colunas['vencedor']).astype(int),
                                (visitantes == colunas['vencedor']).astype(int)
                                ]).sort_index(kind='merge').reset_index(drop=True)
#Aqui acima concatenamos os times de forma alternada e identificamos o vencedor do jogo

#Criando o dataframe
df_final = pd.DataFrame({
    'times': times_alternados,
    'vencedor': vencedores_binarios,
    'resultado': resultado.repeat(2).reset_index(drop=True),  #resultado alternado
    'ano': ano.repeat(2).reset_index(drop=True),
    'casa': coluna_alternada,
    'rodada': rodada.repeat(2).reset_index(drop=True),
    'mandante_Placar': mandante_placar.repeat(2).reset_index(drop=True),
    'visitante_Placar': visitante_placar.repeat(2).reset_index(drop=True)})

In [69]:
#Criar variável binária para identificar se um time está jogando em casa ou fora
num_total = len(df_final)
coluna_alternada = (np.arange(num_total) % 2) ^ 1
coluna_alternada

#Iniciar a coluna 'pontos'
df_final['pontos'] = 0

#Calcular os pontos conquistados pelos times
df_final.loc[(df_final['resultado'] == 0) & (df_final['casa'] == 1), 'pontos'] = 3
df_final.loc[(df_final['resultado'] == 1) & (df_final['casa'] == 0), 'pontos'] = 3
df_final.loc[(df_final['resultado'] == 2), 'pontos'] = 1

In [71]:
#Realiza a soma cumulativa dos pontos
try:
    df_aprov = df_final.groupby(['ano','times'])['pontos'].cumsum()
    df_final['pontos_acum'] = df_aprov
    df_final = df_final.drop(columns = 'pontos')
except KeyError:
    print('A coluna "pontos" já foi removida')

In [73]:
#Calcula os pontos possíveis
df_final['pontos_posv'] = df_final['rodada']*3

#Calcula o aproveitamento
df_final['aproveitamento'] = df_final['pontos_acum'] / df_final['pontos_posv']

df_final['aproveitamento'] = df_final.groupby(['ano','times'])['aproveitamento'].shift(1).fillna(0)

#Remove colunas desnecessárias
df_final = df_final.drop(columns = ['pontos_acum','pontos_posv'])

In [83]:
#Concatenando os dois dataframes
dados_util = pd.concat([df_atualizado, df_final], axis=1)

#Removendo colunas desnecessárias
dados_util = dados_util.drop(columns=['times', 'rodada', 'partida_id', 'index'])

#Removendo colunas duplicadas
dados_util = dados_util.loc[:, ~dados_util.columns.duplicated()]
dados_util

,level_0,ano,rodata,clube,cartao_amarelo,cartao_vermelho,chutes,chutes_no_alvo,escanteios,faltas,impedimentos,passes,posse_de_bola,precisao_passes,vencedor,resultado,casa,mandante_Placar,visitante_Placar,aproveitamento
0,0,2018,1,Cruzeiro,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0,1,1,0,1,0.000000
1,1,2018,1,Gremio,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,1,1,0,0,1,0.000000
2,2,2018,1,Vitoria,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0,2,1,2,2,0.000000
3,3,2018,1,Flamengo,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0,2,0,2,2,0.000000
4,4,2018,1,Santos,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,1,0,1,2,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3795,3795,2022,38,Atletico-MG,2.12,0.00,12.64,2.92,8.96,13.04,1.68,503.16,0.6016,0.8360,1,1,0,0,1,0.495495
3796,3796,2022,38,Palmeiras,2.40,0.08,18.28,7.76,9.56,10.96,2.72,439.40,0.5624,0.8448,1,0,1,3,0,0.630631
3797,3797,2022,38,Internacional,1.68,0.00,14.28,4.92,5.12,10.80,1.92,466.04,0.5368,0.8236,0,0,0,3,0,0.729730
3798,3798,2022,38,Goias,3.60,0.00,11.32,4.20,4.24,16.32,1.76,306.88,0.4100,0.7328,0,1,1,0,4,0.414414


Como próximo passo, irei remover as primeiras 5 e as últimas 5 rodadas de todos os anos. O motivo: **normalmente estas rodadas tendem a ser as mais instáveis em termos de resultados**

In [85]:
#Removendo as primeiras e últimas 5 rodadas
dados_util = dados_util[(dados_util['rodata']>5) & (dados_util['rodata']<34)]

In [89]:
#Salvando dataframe

file_path = 'dados_utilizados.csv'

if not os.path.exists(file_path):
    dados_util.to_csv('dados_utilizados')
    print(f"Arquivo '{file_path}' foi salvo.")
else:
    print(f"Arquivo '{file_path}' já existe e não foi sobrescrito.")

Arquivo 'dados_utilizados.csv' foi salvo.


## Ordenar base de dados

A última etapa do pré-processamento é a de ordenar as variáveis dos times mandante e visitante em apenas uma linha

Logo, vamos dividir nosso dataframe em dois, um com os **dados do time da casa**, outro com os **dados do time visitante**, e realizar a subtração entre os dois (no modelo ‘casa’ menos ‘visitante’).

In [91]:
novos_dados = pd.read_csv('dados_utilizados')

In [95]:
#Move as variáveis imutáveis para o índice
try:
    novos_dados = novos_dados.set_index(['rodata', 'ano', 'resultado']).drop(columns=['Unnamed: 0', 'clube'])
    novos_dados
except KeyError:
    print('Operação já realizada')

In [99]:
#Divisão entre mandantes e visitantes
novos_dados_mandantes = novos_dados.loc[novos_dados['casa'] == 1].rename(columns={'mandante_Placar':'placar'})
novos_dados_visitantes = novos_dados.loc[novos_dados['casa'] == 0].rename(columns={'visitante_Placar':'placar'})

In [103]:
#Faz a subtração entre as duas
df_brasileiro_modelo = novos_dados_mandantes - novos_dados_visitantes
df_brasileiro_modelo

aproveitamento  cartao_amarelo  cartao_vermelho  casa  \
rodata ano  resultado                                                          
6      2018 0                0.200000        0.333333         0.000000     1   
            2                0.200000        1.527778        -0.520833     1   
            0                0.200000       -1.895833         0.062500     1   
            0                0.066667       -1.875000         0.000000     1   
            0                0.066667        1.729167         0.000000     1   
...                               ...             ...              ...   ...   
33     2022 2               -0.234195        1.840000         0.080000     1   
            0               -0.041667        1.080000         0.320000     1   
            1               -0.085685       -1.240000         0.040000     1   
            2               -0.031250       -0.560000         0.000000     1   
32     2022 2               -0.147059       -2.480000         0.000000     1   

                         chutes  chutes_no_alvo  escanteios     faltas  \
rodata ano  resultado                                                    
6      2018 0         -1.020833        1.879401   -1.541667   1.062500   
            2          5.645833        0.750000    1.333333  -1.270833   
            0          2.854167       -0.104167   -1.083333   2.562500   
            0         -5.979167       -2.541667   -1.291667 -10.791667   
            0         -5.888889        1.368056    2.861111   3.152778   
...                         ...             ...         ...        ...   
33     2022 2         -1.520000       -2.240000    1.920000   1.200000   
            0         -3.840000       -1.080000   -1.880000   2.000000   
            1          2.280000        1.025915    2.200000  -3.120000   
            2         -2.600000       -0.880000   -2.920000   2.120000   
32     2022 2          4.480000        2.000000    1.520000  -6.680000   

                       impedimentos  level_0  mandante_Placar      passes  \
rodata ano  resultado                                                       
6      2018 0              2.333333       -1              NaN  -38.145833   
            2              1.131944       -1              NaN  246.763889   
            0              1.000000       -1              NaN  -39.229167   
            0             -1.333333       -1              NaN    9.854167   
            0             -0.152778       -1              NaN  -19.187500   
...                             ...      ...              ...         ...   
33     2022 2              0.520000       -1              NaN -156.760000   
            0              0.720000       -1              NaN   -6.000000   
            1              0.760000       -1              NaN   60.360000   
            2              1.040000       -1              NaN -174.240000   
32     2022 2              0.280000       -1              NaN  236.240000   

                       placar  posse_de_bola  precisao_passes  vencedor  \
rodata ano  resultado                                                     
6      2018 0               1      -0.027083         0.006250         1   
            2               0       0.239792         0.090417         0   
            0               3      -0.013750         0.016458         1   
            0               1      -0.086667         0.022708         1   
            0               1      -0.004861        -0.029028         1   
...                       ...            ...              ...       ...   
33     2022 2               0      -0.113600        -0.058400         0   
            0               1       0.010400        -0.020400         1   
            1              -1       0.036800         0.032000        -1   
            2               0      -0.197600        -0.083200         0   
32     2022 2               0       0.166800         0.086800         0   

                       visitante_Placar  

In [105]:
#Removemos as colunas de placar e voltamos as variáveis ano e rodada para o dataframe
df_brasileiro_modelo = df_brasileiro_modelo.drop(
    columns=['mandante_Placar', 'visitante_Placar', 'placar', 'casa']).reset_index(
            level=[1, 2])

In [111]:
df_brasileiro_modelo.to_csv('df_modelo')